# Function Tests

This notebook tests `countmatch_common.py`, `countmatch_teps.py` and `countmatch_bagheri.py`.

In [1]:
%matplotlib inline
import sys
sys.path.append('../')
import importlib
import matplotlib.pyplot as plt
import numpy as np
import knowyourdata as kyd

import pandas as pd
from traffic_prophet import cfg
import pathlib, os
import configparser

from traffic_prophet import connection
from traffic_prophet.countmatch import reader
from traffic_prophet.countmatch import growthfactor as gf
from traffic_prophet.countmatch import neighbour

defaultcolours = plt.rcParams['axes.prop_cycle'].by_key()['color']

filepath = pathlib.Path.home().joinpath('.charlesconfig')
if os.path.isfile(filepath):
    vol_conn = connection.Connection(filepath, 'POSTGRES',
                                     'czhu.btp_centreline_daily_counts')
    ll_conn = connection.Connection(filepath, 'POSTGRES',
                                    'czhu.btp_centreline_lonlat')
    config = configparser.RawConfigParser()
    config.read(filepath.as_posix())
    MAPBOX_TOKEN = config['MAPBOX']['token']
    PLOTLY_USER = config['PLOTLY']['user']
    PLOTLY_KEY = config['PLOTLY']['key']
else:
    filepath = pathlib.Path.home().joinpath('cf.txt')
    vol_conn = connection.Connection(filepath, 'localpg',
                                     'prj_vol.btp_centreline_daily_counts')
    ll_conn = connection.Connection(filepath, 'localpg',
                                    'gis.btp_centreline_lonlat')
    config = configparser.RawConfigParser()
    config.read(filepath.as_posix())
    MAPBOX_TOKEN = config['mapbox']['token']
    PLOTLY_USER = config['plotly']['user']
    PLOTLY_KEY = config['plotly']['apikey']

In [2]:
%%time
rdr_teps = reader.Reader(vol_conn)
rdr_teps.read()
gf.get_growth_factors(rdr_teps)

CPU times: user 1min 55s, sys: 2.62 s, total: 1min 57s
Wall time: 2min 9s


In [3]:
ptc_ids = np.unique(np.abs(list(rdr_teps.ptcs.keys())))
nb = neighbour.NeighbourLonLatEuclidean(ll_conn, 20, ptc_ids)
%time nb.find_neighbours()

CPU times: user 19.8 s, sys: 328 ms, total: 20.1 s
Wall time: 20.1 s


In [4]:
%%time
rdr_bag = reader.Reader(vol_conn)
rdr_bag.read()
gf.get_growth_factors(rdr_bag)

CPU times: user 2min 2s, sys: 2.83 s, total: 2min 4s
Wall time: 2min 14s


## TEPs-like CountMatch

Testing the `countmatch_teps` module.

In [5]:
import countmatch_teps as cmt
importlib.reload(cmt)

<module 'countmatch_teps' from '/mnt/c/Users/czhu5/Documents/Ubuntu/GitHub/bdit_traffic_prophet/development_notebooks/countmatch_teps.py'>

In [6]:
teps_aadt_estimates = cmt.estimate_aadts(
    rdr_teps, nb, 2019, n_neighbours=5, single_direction=True, progress_bar=True)

In [7]:
import countmatch_common as cmc
importlib.reload(cmc)

<module 'countmatch_common' from '/mnt/c/Users/czhu5/Documents/Ubuntu/GitHub/bdit_traffic_prophet/development_notebooks/countmatch_common.py'>

In [8]:
ptc_estimates = cmc.aadt_estimator_for_ptcs(rdr_teps, 2019, override_growth_factor=False)

In [9]:
teps_aadt_estimates = cmc.combine_estimators(teps_aadt_estimates, ptc_estimates)

In [10]:
teps_aadt_estimates

,Count ID,PTC ID,Closest Year,D_ij,AADT Estimate_STTC,AADT Estimate_PTC,AADT Estimate
0,-175,-445884,2010,0.976424,2208.113549,NaN,2208.113549
1,175,446012,2010,0.915809,1818.715350,NaN,1818.715350
2,-201,-443987,2006,0.937714,339.977238,NaN,339.977238
3,201,445280,2006,0.984605,267.842517,NaN,267.842517
4,-202,-443975,2006,0.980700,213.250136,NaN,213.250136
5,202,445280,2006,0.984605,243.810970,NaN,243.810970
6,-214,-1022,2011,0.964907,3281.430923,NaN,3281.430923
7,-229,-443987,2006,0.937714,357.870777,NaN,357.870777
8,229,444782,2006,0.995233,310.040831,NaN,310.040831
9,-256,-443975,2006,0.962607,15733.294198,NaN,15733.294198


## Hybrid Count Module

Testing `countmatch_hybrid.py`.

In [14]:
import countmatch_hybrid as cmh
importlib.reload(cmh)

<module 'countmatch_hybrid' from '/mnt/c/Users/czhu5/Documents/Ubuntu/GitHub/bdit_traffic_prophet/development_notebooks/countmatch_hybrid.py'>

In [15]:
bag_aadt_estimates = cmh.estimate_aadts(rdr_bag, nb, 2019, progress_bar=True)

In [16]:
bag_aadt_estimates.head()

,Count ID,Nearest PTCS,Errors,PTC ID,Growth Factor,Most Recent Year,AADT Estimate
0,-175,"[-1022, -444637, -890, -445309, -445884]","[0.5608327532405594, 0.8307061415303842, 0.634...",-1022,1.135230,2010,5378.846957
1,175,"[446012, 1033, 444516, 1003, 444782]","[0.48182483033236795, 0.4179452872922771, 0.20...",444516,1.271847,2010,12889.208626
2,-201,"[-445309, -445884, -444637, -443975, -443987]","[0.0029596013716408163, 0.0, 0.001936558982980...",-445884,0.909007,2006,74.947454
3,201,"[445280, 444782, 444403, 444516, 446012]","[0.000271132236083406, 0.0, 0.0, 0.0, 7.335767...",444782,0.984669,2006,157.838043
4,-202,"[-445309, -445884, -444637, -443975, -443987]","[1.0984804229158482e-05, 0.0, 0.00291553627916...",-445884,0.909007,2006,49.870941


In [17]:
bag_aadt_estimates = cmc.combine_estimators(bag_aadt_estimates, ptc_estimates)

In [18]:
bag_aadt_estimates

,Count ID,Nearest PTCS,Errors,PTC ID,Growth Factor,Most Recent Year,AADT Estimate_STTC,AADT Estimate_PTC,AADT Estimate
0,-175,"[-1022, -444637, -890, -445309, -445884]","[0.5608327532405594, 0.8307061415303842, 0.634...",-1022,1.135230,2010,5378.846957,NaN,5378.846957
1,175,"[446012, 1033, 444516, 1003, 444782]","[0.48182483033236795, 0.4179452872922771, 0.20...",444516,1.271847,2010,12889.208626,NaN,12889.208626
2,-201,"[-445309, -445884, -444637, -443975, -443987]","[0.0029596013716408163, 0.0, 0.001936558982980...",-445884,0.909007,2006,74.947454,NaN,74.947454
3,201,"[445280, 444782, 444403, 444516, 446012]","[0.000271132236083406, 0.0, 0.0, 0.0, 7.335767...",444782,0.984669,2006,157.838043,NaN,157.838043
4,-202,"[-445309, -445884, -444637, -443975, -443987]","[1.0984804229158482e-05, 0.0, 0.00291553627916...",-445884,0.909007,2006,49.870941,NaN,49.870941
5,202,"[445280, 444782, 444403, 444516, 446012]","[2.8935805908725606e-06, 0.0, 0.0, 0.0, 0.0077...",444782,0.984669,2006,146.990007,NaN,146.990007
6,-214,"[-1022, -890, -444637, -445884, -445309]","[0.06788079244670922, 0.0029656457895476957, 0...",-890,1.038726,2011,3809.125278,NaN,3809.125278
7,-229,"[-445309, -445884, -444637, -443975, -443987]","[0.0029596013716407924, 0.0, 0.001936558982980...",-445884,0.909007,2006,78.842754,NaN,78.842754
8,229,"[445280, 444782, 444403, 444516, 446012]","[0.000271132236083406, 0.0, 0.0, 0.0, 7.335767...",444782,0.984669,2006,180.716896,NaN,180.716896
9,-256,"[-445309, -445884, -444637, -443975, -443987]","[5.074858420381781e-05, 0.0, 0.006524063642341...",-445884,0.909007,2006,3637.407741,NaN,3637.407741


## Bagheri Count Module

Lastly, we test `countmatch_bagheri.py`, which most closely resembles Bagheri et al.

In [19]:
import countmatch_bagheri as cmb
importlib.reload(cmb)

<module 'countmatch_bagheri' from '/mnt/c/Users/czhu5/Documents/Ubuntu/GitHub/bdit_traffic_prophet/development_notebooks/countmatch_bagheri.py'>

In [20]:
# Process nearest PTC comparisons to estimate AADT for each STTC.
cmb.get_aadt_estimate_for_sttc(
    rdr_bag.sttcs[-175], rdr_bag.ptcs, nb, 2019, cmb.estimate_mse)

(-175,
 [-1022, -444637, -890, -445309, -445884],
 [0.5608327532405594,
  0.68604128629566,
  0.6339420489270072,
  0.559453533374608,
  0.46442693081405684],
 -445884,
 0.9090066894109187,
 2010,
 709.1793723945789)

In [21]:
cmh.get_aadt_estimate_for_sttc(
    rdr_bag.sttcs[-175], rdr_bag.ptcs, nb, 2019)

(-175,
 [-1022, -444637, -890, -445309, -445884],
 [0.5608327532405594,
  0.8307061415303842,
  0.6344208625386252,
  0.6842551418750419,
  0.7747454246136338],
 -1022,
 1.135230414860393,
 2010,
 5378.846957103764)

In [22]:
tc = rdr_bag.sttcs[-175]
ptcs = rdr_bag.ptcs
want_year = 2019
n_neighbours=5
single_direction=True,

# Find nearest neighbours in same direction.
neighbour_ptcs = cmc.get_neighbours(
    tc, ptcs, nb, n_neighbours=n_neighbours,
    single_direction=single_direction)

# Determine minimum MSE tables for closest neighbouring PTCs.
mses = []
for ptc in neighbour_ptcs:
    ptc_match = cmh.get_ptc_match(tc, ptc)
    # growth_factor = (override_growth_factor if override_growth_factor
    #                  else ptc.growth_factor)
    mses.append((ptc.count_id,) +
                cmh.estimate_mse(ptc_match, ptc, want_year,
                             cmc.get_citywide_growth_factor(rdr_bag)))
# Retrieve `ptc_match` table of minimum MSE PTC.
min_countid, mmse_ptc_match = min(mses, key=lambda x: x[-1])[0:2]

In [23]:
mses[0][1]

,Year,Date,Daily Count,Day of Week,Month,Closest Year,D_ijd,DoM_ijd,MADT_est
0,2006,2006-08-15,171.0,1,8,2016,0.951817,0.968189,220.411747
1,2010,2010-06-09,1855.0,2,6,2016,0.925953,0.962942,2177.623663


In [24]:
mses[1][1]

,Year,Date,Daily Count,Day of Week,Month,Closest Year,D_ijd,DoM_ijd,MADT_est
0,2006,2006-08-15,171.0,1,8,2010,0.975602,1.005890,228.994586
1,2010,2010-06-09,1855.0,2,6,2010,0.991796,0.984848,2227.162266


In [25]:
# Process nearest PTC comparisons to estimate AADT for each STTC.
cmb.get_aadt_estimate_for_sttc(
    rdr_bag.sttcs[-175], rdr_bag.ptcs, nb, 2019, cmb.estimate_mse,
    override_growth_factor=cmc.get_citywide_growth_factor(rdr_bag))

(-175,
 [-1022, -444637, -890, -445309, -445884],
 [0.6984172544782186,
  0.8374002130428739,
  0.7795400789582994,
  0.69686364325305,
  0.5902844916985741],
 -445884,
 1.0222564979935564,
 2010,
 2040.2871790117917)